DR Guia passo a passo para construir um modelo de rede neural profunda com Keras para prever Preços do Airbnb em Nova York e implantação como API REST usando Flask

Este guia permitirá implantar um modelo de aprendizado de máquina começando do zero. Aqui estão as etapas você vai cobrir:

• Defina seu objetivo
• Carregar dados
• Exploração de dados
• Preparação de dados
• Construa e avalie seu modelo
• Salve o modelo
• Construir API REST
• Implantar em produção

### Definir objetivo/meta

Obviamente, você precisa saber por que precisa de um modelo de aprendizado de máquina (ML).Conhecer o objetivo fornece insights sobre:

• O ML é a abordagem certa?
• De quais dados eu preciso?
• Qual será a aparência de um “bom modelo”? Quais métricas posso usar?
• Como resolvo o problema agora? Quão precisa é a solução?
• Quanto custará manter este modelo em funcionamento?

Em nosso exemplo, estamos tentando prever o preço de listagem do Airbnb³² por noite em Nova York. Nosso objetivo é claro - dados alguns dados, queremos que nosso modelo preveja quanto custará alugar um determinado imóvel
por noite.

### Carregar dados

Os dados vêm do Airbnb Open Data e estão hospedados no Kaggle³³

Desde 2008, hóspedes e anfitriões usam o Airbnb para ampliar as possibilidades de viagem e apresentar uma maneira mais única e personalizada de experimentar o mundo. Este conjunto de dados descreve
a atividade de listagem e métricas em NYC, NY para 2019.

#### Configuração 

Começaremos com várias importações e definindo uma semente aleatória para reprodutibilidade:

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 16, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
df = pd.read_csv('data_airbnb/airbnb_nyc.csv')